# Tabela Posição Estoque Work

## Imports

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

## Análise Inicial

In [0]:
posicao_estoque = spark.sql("SELECT * FROM raw.posicao_estoque_raw")
posicao_estoque.display()

## Casting

In [0]:
posicao_estoque = posicao_estoque.withColumn('DATA', to_date('DATA', 'yyyy-MM-dd')) \
    .withColumn('CMV', regexp_replace('CMV', ',', '.').cast(DoubleType())) \
    .withColumn('ESTOQUE_VALOR', regexp_replace('ESTOQUE_VALOR', ',', '.').cast(DoubleType())) \
    .withColumn('ESTOQUE_QTDE', regexp_replace('ESTOQUE_QTDE', ',', '.').cast(IntegerType())) \
    .withColumn('VENDA_VALOR', regexp_replace('VENDA_VALOR', ',', '.').cast(DoubleType())) \
    .withColumn('VENDA_QTDE', regexp_replace('VENDA_QTDE', ',', '.').cast(IntegerType())) \
    .withColumn('MARGEM_FINAL', regexp_replace('MARGEM_FINAL', ',', '.').cast(DoubleType())) \
    .withColumn('VENDA_LIQUIDA_FINAL', regexp_replace('VENDA_LIQUIDA_FINAL', ',', '.').cast(DoubleType())) \
    .withColumn('PRODUTO', regexp_replace('PRODUTO', ',', '.').cast(IntegerType())) \
    .withColumn('LOJA', regexp_replace('LOJA', ',', '.').cast(IntegerType()))

## Renomeando e Organizando as Colunas

In [0]:
posicao_estoque = posicao_estoque.withColumnRenamed('DATA', 'data_extracao') \
    .withColumnRenamed('CMV', 'custo_mercadoria_vendida') \
    .withColumnRenamed('ESTOQUE_VALOR', 'estoque_valor') \
    .withColumnRenamed('ESTOQUE_QTDE', 'estoque_quantidade') \
    .withColumnRenamed('VENDA_VALOR', 'venda_total') \
    .withColumnRenamed('VENDA_QTDE', 'venda_quantidade') \
    .withColumnRenamed('MARGEM_FINAL', 'margem') \
    .withColumnRenamed('VENDA_LIQUIDA_FINAL', 'venda_liquida') \
    .withColumnRenamed('PRODUTO', 'cod_produto') \
    .withColumnRenamed('LOJA', 'cod_loja')

In [0]:
posicao_estoque = posicao_estoque[['data_extracao',
 'cod_loja',
 'cod_produto',
 'custo_mercadoria_vendida',
 'estoque_valor',
 'estoque_quantidade',
 'venda_total',
 'venda_quantidade',
 'margem',
 'venda_liquida']]

In [0]:
posicao_estoque.display()

## Removendo linhas duplicadas

In [0]:
posicao_estoque = posicao_estoque.coalesce(1).dropDuplicates()

## Salvando Dados

In [0]:
posicao_estoque.write \
    .option('path', 'mnt/work/posicao_estoque_main_work') \
    .partitionBy(['data_extracao', 'cod_loja', 'cod_produto']) \
    .mode('overwrite') \
    .saveAsTable('work.posicao_estoque_main_work')

## Otimizando

In [0]:
%sql
OPTIMIZE work.posicao_estoque_main_work

path,metrics
dbfs:/mnt/dev/posicao_estoque_main_dev,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 100, null, 0, 100, 100, true, 0, 0, 1679945246112, 1679945248585, 8, 0, null)"
